In [1]:
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders.text import TextLoader
from langchain_community.embeddings.gigachat import GigaChatEmbeddings
from langchain_community.chat_models import GigaChat
from langchain.embeddings import HuggingFaceEmbeddings
from typing import Any, Coroutine, List
from dotenv import load_dotenv
from itertools import chain
from os import getenv

load_dotenv()

True

In [2]:
docs_list = ["documents/about.txt", "documents/number_phones.txt", "documents/rectors.txt", "documents/iisigt.txt", "documents/checking.txt"]
loader = map(lambda x: TextLoader(file_path=x), docs_list)
documents = list(map(lambda x: x.load(), loader))

flattened_documents = list(chain.from_iterable(documents))

In [3]:
class HuggingFaceE5Embeddings(HuggingFaceEmbeddings):
    def embed_query(self, text: str) -> List[float]:
        text = f"query: {text}"
        return super().embed_query(text)

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        texts = [f"passage: {text}" for text in texts]
        return super().embed_documents(texts)

    async def aembed_query(self, text: str) -> Coroutine[Any, Any, List[float]]:
        text = f"query: {text}"
        return await super().aembed_query(text)

    async def aembed_documents(
        self, texts: List[str]
    ) -> Coroutine[Any, Any, List[List[float]]]:
        texts = [f"passage: {text}" for text in texts]
        return await super().aembed_documents(texts)


In [4]:
embedding = HuggingFaceE5Embeddings(model_name="intfloat/multilingual-e5-base")


/home/yazop/Desktop/bachelor_diploma/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
#embedding = GigaChatEmbeddings(credentials=getenv("WORK_CREDENTIAL"), verify_ssl_certs=False, scope=getenv("WORK_SCOPE"))

In [9]:
llm = GigaChat(credentials=getenv("CREDENTIAL"), verify_ssl_certs=False, scope=getenv("SCOPE"), profanity=False)

'profanity' field is deprecated. Use 'profanity_check' instead.


In [10]:
db = FAISS.from_documents(
    flattened_documents,
    embedding
)

In [17]:
question = "К кому обратиться при возникших вопросах с заселением в общежитие?"

In [18]:
db.similarity_search(question, k=2)


[Document(page_content='ПОРЯДОК ЗАСЕЛЕНИЯ ПЕРВОКУРСНИКОВ В ОБЩЕЖИТИЕ\n\n1. Подготовьте документы, необходимые для поселения:\n\n-     Паспорт;\n\n-     Сертификат о профилактических прививках;\n\n-     Результат флюорографического обследования за последний год;\n\n-     Медицинская справка 086/у;\n\n-     Для несовершеннолетних первокурсников (в случае, если первокурсник приезжает самостоятельно) – нотариально заверенное согласие родителей на право заключения договора найма жилого помещения в общежитиях РГГМУ и право оформления временной регистрации.\n\n2. Ознакомьтесь со стоимостью проживания:\n\n-   В настоящий момент общежитие для студентов бюджетной основы стоит 1525 и в районе 4645 для контрактной основы.\n\n3. Ознакомьтесь со сроками поселения\n\nПоселение первокурсников 2023 года начнется с 23.08.2023. Оформление документов и заключение договоров возможно по будним дням с 10.00 до 17.00.\n\n4. Ознакомьтесь с распределением первокурсников по общежитиям\n\n5. Явитесь в общежитие, 

In [19]:
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=db.as_retriever())

In [20]:
qa_chain({"query": question})

{'query': 'К кому обратиться при возникших вопросах с заселением в общежитие?',
 'result': 'При возникших вопросах с заселением в общежитие рекомендуется обратиться к начальнику Управления по работе с общежитиями Тихомирову Сергею Николаевичу. Его контактный телефон указан в предоставленной информации.'}